<a href="https://colab.research.google.com/github/mldurga/easydl/blob/master/_notebooks/2021-10-14-super_convergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topology of Loss function

Nothing is more explainable than this 

<blockquote class="twitter-tweet"><p lang="en" dir="ltr"><a href="[tweet_url]">pic.twitter.com/xBhagsGlXM</a></p><a href="https://twitter.com/zozuar/status/1443012484189888515?s=20"></a></blockquote>

<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>


## Introduction
Until publication of the papers [1](https://arxiv.org/pdf/1506.01186.pdf),[2](https://arxiv.org/pdf/1803.09820.pdf),[3](https://arxiv.org/pdf/1708.07120.pdf) by Mr Leslie Smith, finding learning rate of neural networks training is largely a black art and often requires extensive practice and expertise to set good hyperparameters. Mr Smith's contribution greatly helped reserach community on overall understanding of the topology of loss function. Unlike typical Machine Learning papers, Smith's Papers are very much approachable and invaluable. I have tried to reproduce fastai source code of implementation of 1cycle policy inspired from Smith's papers.

##Learning rate (LR) and Weight updataion
In neural networks training, loss function (simply distance between target value and predicted value) must be decreased with each iteration. SGD - stochastic gradient descent is the method to achieve the above stated goal. If you can observe the above twitter card, loss function topology will be more or less have same features. Identifying global minimum of that topology is the sole aim of any Machine learning practitioner. However, training gets trapped over local minima and model underperforms. This is where Smiths ideas catapulted training neural networks to the next level. 

![image](https://user-images.githubusercontent.com/19243618/137344851-460b2d6d-30a4-4f31-b4fc-6382581dd87d.png)

```w.new=w.old - gradient(w.old) * lr```

above image gives general idea of Lerning Rate value impact on convergence of loss function. weights will get updated for each iteration at the rate of choosen LR. Low LR can have effect of trapping at [saddle](https://en.wikipedia.org/wiki/Saddle_point) points and local minima, High LR can immediately be diverged and loss value increases greatly. Until Smith's papers, setting right LR has been a great challenge.

## Cyclical Learning Rate (CLR)
Conventionally LR is a fixed value and tends to be decreased over a period of training until a good accuracy is obtained. Smith introduced novel technique of varying learning rate in cyclical fashion resulted in huge gains in accuracy in few iterations compared to the conventional practices.

![cifar_LR](https://user-images.githubusercontent.com/19243618/137067445-bef503ca-dc77-447b-88e7-ba6126908b2d.jpg)

the CLR approach has clearly outperformed other techniques of setting LR as hown in figure. CLR achieved same accuracy within 20k iterations compared to 80k iterations of other techniques.


![image](https://user-images.githubusercontent.com/19243618/137253367-2cf2c603-85ef-4c10-abfc-24555536025e.png)

LR has to be varied between two bounds called maximum bound and minimum bound. These can be found from LR range test, which is identifying minimum and maximum learning rate values where accuracy is actually increasing, by doing few iteratrions of training. Due to simplicity, triangular learning rate policy was adopted for most of the cases. following terminology helps in understanding setting of learning rate.
 


*  epoch: one pass through of the entire dataset
*  batch size: No of images in a mini batch
*  iteration: training over a mini batch

Lets take 50,000 images to be trained and with a 100 images per minibatch 


*   No of iterations per epoch => (50000/100) =500
*   stepsize : Generally 2 times epoch, so every 1000 iterations we have one increaing learning rate and next 1000 iterations we have one decreasing learning rate, this becomes one cycle

Smiths experiments shows that generally 3 cycles have trained the network within short iterations compared to conventional LR methods. However 1cycle [paper](https://arxiv.org/pdf/1708.07120.pdf) in 2018 stated that one cycle is sufficient to train the network when very large learning rates are used.




## Super-Convergence
Cyclic learning rates coupled with large values of learning rates leads to convergence much faster than standard training methods. The large learning rates allows training to jump over the local minimas and will find a way to global minima in much less time. Cyclical nature of the learning rates enables training to first slowly converge then moves at faster speeds to traverse the valley and again slows the pace to finally converge to final accuracy level. 

![image](https://user-images.githubusercontent.com/19243618/137189466-0b3eca30-9ad9-4e5b-9cf6-6b1c4deb5be9.png)


Above images are from super-convergence paper. Figure (a) clearly shows the dominance of Super-Convergence approach denoted by CLR. With CLR range 0.1-3 (Learning rate bounds) accuracy obtained 92.4% within 10k iterations compared to 80k iterations of normal training methods on CIFAR-10 Dataset, ResNet-56 architecture.

To reiterate author words "Always use one cycle that is smaller than the total number of iterations/epochs and allow the learning rate to decrease several orders of magnitude less than the initial learning rate for the remaining iterations". This is called 1cycle policy or Super-Convergence. Since 1cycle policy have regularisation effect on overall training process, care must be taken to balance the remianing regularisation techniques like weight-decay, batch size, drop out etc.. depending on the dataset.




The maximum learnng rate will be obtained from the LR range test, in which maximum learning rate at which accuracy stops increasing will be taken as max LR and minimum LR will be factor of 10 to 20 less than the max LR. 

### Momentum
According to Leslie papers, Momentum is as important as learning rate for training the network. The value of momentum observed to be decreasing while learning rate value is increasing in a stepsize of 1cycle policy. Momentum is an attempt to maintain consistent direction of SGD (stochastic Gradient Direction). If we set momentum at 90%, it means we will take 90% of the previous direction and 10% of the new direction. Intuitiviely we can look at moomentum as a process of giving more weight to new gradients or directions to find flatter area. Leslie smith suggest to use momentum values between 0.85 and 0.95 for training purpose.

## ```fit_one_cycle``` and ```fine_tune``` of fastai
The above paper has been implemented in fastai with certain modifications. Source code of fastai related to these functions were presented here. Since this library is designed with lot of functionality, and all of them cant be reproduced here, interested can explore the same in [github](https://github.com/fastai/fastai/tree/master/fastai) for entire source code. 

In [ ]:
#| include: false
!pip install fastbook -Uqq
import fastbook
from fastai.vision.all import *

     |████████████████████████████████| 720 kB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 36.5 MB/s 
     |████████████████████████████████| 186 kB 44.7 MB/s 
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
     |████████████████████████████████| 51 kB 302 kB/s 


Lets explore bit by bit. First we will look at ```fine_tune()``` then drill down gradully

In [ ]:
def fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tune with `Learner.freeze` for `freeze_epochs`, then with `Learner.unfreeze` for `epochs`, using discriminative LR."
    self.freeze()
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    self.unfreeze()
    self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)

```fine_tune``` initially freezes pretrained model weights and trains using ```fit_one_cycle``` with one epoch to enable random weights in head to adjust to new dataset. After unfreeze entire network will be trained with same ```fit_one_cycle``` method with choosen no of epochs.

### Exploring source code in fastai library can give good insights

![image](https://user-images.githubusercontent.com/19243618/137338003-dfdbf632-4ea7-444b-af40-e68319d076d8.png)

```pct_start``` is percentage of overall iterations where LR is increaing. if ```pct_start``` is 0.25 that means LR is increasing up to 25 % of iterations and remaining 75 % iterations LR is decreasing. other plot is of momentum, its values varies inverse to the LR and range is between 0.95 to 0.85. However the choice of the momentum values will be depend on the dataset and architectures.

Other Helper functions in source code

![image](https://user-images.githubusercontent.com/19243618/137340342-934f87a6-e379-438f-a661-5fe39b6af111.png)

## Credits


*   Leslie Smith Papers [1](https://arxiv.org/pdf/1506.01186.pdf),[2](https://arxiv.org/pdf/1803.09820.pdf),[3](https://arxiv.org/pdf/1708.07120.pdf)
*   Fastai library [github](https://github.com/fastai/fastai/tree/master/fastai)
*   Slvain Gugger [post](https://sgugger.github.io/the-1cycle-policy.html)
*   [stack over flow](https://stackoverflow.com/questions/56482528/what-is-momentum-in-machine-learning/56482888)

